In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


In [10]:
# Function to parse IGC file (simplified example)
def parse_igc(file_path):
    # Read IGC file (this is a placeholder parser; refine it as per your IGC structure)
    with open(file_path, 'r') as file:
        lines = file.readlines()

    data = {'time': [], 'latitude': [], 'longitude': [], 'altitude': []}

    for line in lines:
        if line.startswith('B'):  # Flight data line
            time = line[1:7]  # HHMMSS
            lat = int(line[7:14]) / 100000.0
            lon = int(line[15:23]) / 100000.0
            alt = int(line[25:30])

            data['time'].append(time)
            data['latitude'].append(lat)
            data['longitude'].append(lon)
            data['altitude'].append(alt)

    return pd.DataFrame(data)


In [11]:
# Load the flight data
data = parse_igc("data/2024-08-03 09_11_21.igc")
data.head()

,time,latitude,longitude,altitude
0,091121,40.01887,32.19713,1142
1,091122,40.01887,32.19713,1142
2,091123,40.01887,32.19713,1142
3,091124,40.01887,32.19713,1142
4,091125,40.01886,32.19713,1142


In [ ]:
# Feature engineering
# Converting time to seconds for numerical calculations
data['time'] = data['time'].apply(lambda x: int(x[:2]) * 3600 + int(x[2:4]) * 60 + int(x[4:]))
data['time_diff'] = data['time'].diff().fillna(0)
data['altitude_diff'] = data['altitude'].diff().fillna(0)

# Features and target
X = data[['time_diff', 'latitude', 'longitude']]
y = data['altitude_diff']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Save predictions with test data
results = X_test.copy()
results['actual'] = y_test
results['predicted'] = y_pred

# Save the results to a CSV file
results.to_csv('predictions.csv', index=False)

print("Predictions saved to 'predictions.csv'")
